In [5]:
from pymongo import MongoClient
import datetime
import re

In [6]:
client = MongoClient()
titles = client.ck2.titles

In [32]:
titles.delete_many({})

title = {}
holder_list = []
holder_dict = {}
curr_title = None
holder = None
start_date = None
end_date = None
holder_type = None
most_recent_date = None
name = None
odd_title = False

with open("../WorkInProgress2/title.txt", encoding="utf-8") as titlefile:
    for line in titlefile:
        if len(re.findall(r"^\s*([a-z]{1}_{1}[_a-z0-9\u00C0-\u017F]*)=\s*", line)) > 0:
            if start_date is not None:
                holder_dict["start_date"] = start_date
                holder_dict["holder"] = holder
                if holder_type is not None:
                    holder_dict["type"] = holder_type
                holder_dict['ruling_as_of'] = datetime.datetime(int(1116), int(1), int(1))
                holder_list.append(holder_dict)            
                if "title" in title.keys():
                    if name is not None:
                        title["name"] = name
                        name = None
                    title["holders"] = holder_list
                
                titles.insert_one(title)
                
                title = {}
                holder_list = []
                holder_dict = {}
                curr_title = None
                holder = None
                start_date = None
                end_date = None
                holder_type = None
                most_recent_date = None
                name = None
                odd_title = False
            
            title["title"] = re.findall(r"^\s*([a-z]{1}_{1}[_a-z0-9\u00C0-\u017F]*)=\s*", line)[0]
            
            if any(char.isdigit() for char in title["title"]):
                odd_title = True
            
        #If a date is found
        if len(re.findall(r"^\s*([0-9]+.[0-9]+.[0-9]+)=", line)) > 0:
            this_date = re.findall(r"^\s*([0-9]+).([0-9]+).([0-9]+)=", line)
            date_to_use = datetime.datetime(int(this_date[0][0]), int(this_date[0][1]), int(this_date[0][2]))
            
            #Getting past the first run when no date is set
            if start_date is None:
                start_date = date_to_use
                holder_dict["start_date"] = start_date
            else:
                most_recent_date = date_to_use
                
                
                
        #Id of the title holder, historical (for rulers before the game start)
        if len(re.findall(r"^\s*holder=([0-9]+)\s*", line)) > 0:
            
            if most_recent_date is not None:
                holder_dict["start_date"] = start_date
                holder_dict["holder"] = holder
                holder_dict["end_date"] = most_recent_date
                if holder_type is not None:
                    holder_dict["type"] = holder_type
                
                holder_list.append(holder_dict.copy())
                
                holder_dict = {}
                start_date = most_recent_date
                end_date = None
                holder = None
                holder_type = None
            
            if start_date is not None:
                holder = int(re.findall(r"^\s*holder=([0-9]+)\s*", line)[0])
        
        #Where title has lapsed - no holder
        if len(re.findall(r"^\s*(holder=\"0\")\s*", line)) > 0:            
            
            if most_recent_date is not None:
                holder_dict["start_date"] = start_date
                holder_dict["holder"] = holder
                holder_dict["end_date"] = most_recent_date
                if holder_type is not None:
                    holder_dict["type"] = holder_type
                
                holder_list.append(holder_dict.copy())
                
                holder_dict = {}
                start_date = most_recent_date
                end_date = None
                holder = None
                holder_type = None
            
            holder = 0
              
        
        #Id of the title holder (where who and type are set)
        elif len(re.findall(r"^\s*who=([0-9]+)\s*", line)) > 0:
            
            if most_recent_date is not None:
                holder_dict["start_date"] = start_date
                holder_dict["holder"] = holder
                holder_dict["end_date"] = most_recent_date
                if holder_type is not None:
                    holder_dict["type"] = holder_type
                
                holder_list.append(holder_dict.copy())
                
                holder_dict = {}
                start_date = most_recent_date
                end_date = None
                holder = None
                holder_type = None
            
            holder = int(re.findall(r"^\s*who=([0-9]+)\s*", line)[0])
            
            
        #type (election, inherited etc)
        elif len(re.findall(r"^\s*type=(\w+)\s*", line)) > 0:
            if holder is not None:
                holder_type = re.findall(r"^\s*type=(\w+)\s*", line)[0]
        
        #some titles are generated during the game (id d_dyn_123456) such as revolts and patrician families.  
        #These contain a number of "name=" rows (wars etc). The row that is wanted is may contain unicode
        elif len(re.findall(r"^\s*name=\"([a-zA-Z\s\u00C0-\u017F]*)\"\s*", line)) > 0:
            if "title" in title.keys() is not None:
                if odd_title and len(holder_list) <= 0:
                    name = re.findall(r"^\s*name=\"([a-zA-Z\s\u00C0-\u017F]*)\"\s*", line)[0]
                    odd_title = False
            
                 
            
            
if start_date is not None:
    holder_dict["start_date"] = start_date
    holder_dict["holder"] = holder
    if holder_type is not None:
        holder_dict["type"] = holder_type
    holder_dict['ruling_as_of'] = datetime.datetime(int(1116), int(1), int(1))
    holder_list.append(holder_dict)            
if "title" in title.keys():
    if name is not None:
        title["name"] = name
    title["holders"] = holder_list
    name = None
    titles.insert_one(title)
print("Fin")

Fin
